In [4]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import *
from sympy import *
import copy

In [5]:
def Time( radius, mass ):
    return simplify( 1 - ( 2.0 * mass ) / radius )

In [6]:
def Radius( radius, mass ):
    return simplify( Time( radius, mass ) ** ( -1 ) )

In [7]:
def Theta( radius ):
    return ( radius ** 2 )

In [8]:
def Phi( radius, theta ):
    return ( ( radius **2 ) * ( sin( theta ) ** 2 ) )

In [9]:
def ShwarzchildMetric( mass, radius, theta ):
    return [ [ -Time( radius, mass ), 0.0, 0.0, 0.0 ], 
               [ 0.0, Radius( radius, mass ), 0.0, 0.0 ], 
               [ 0.0, 0.0, Theta( radius ), 0.0 ], 
               [ 0.0, 0.0, 0.0, Phi( radius, theta ) ] ]

In [10]:
time = Symbol( 'time' )
radius = Symbol( 'radius' ) 
theta = Symbol( 'theta' )
phi = Symbol( 'phi' )

In [22]:
symboles = [ time, radius, theta, phi ]
display = [ 't', 'r', 'theta', 'phi' ]#[ u't', u'r', u'u0398', u'u03D5' ]
def CoordinateString( a, b, c ):
    return display[ a ] + u', ' + display[ b ] + u', ' + display[ c ] + u': '

In [12]:
def DeriveCoordinate( metric, x, y, withRespectTo ):
    if( type( metric[ x ][ y ] ) != float ):
        metric[ x ][ y ] += metric[ x ][ y ].diff( withRespectTo )
    else:
        metric[ x ][ y ] += 0.0
    return metric

In [63]:

def IntermediateConnection( mass, radius, theta, a, b, c, symboles ):
    metric0 = ShwarzchildMetric( mass, radius, theta )
    metric1 = ShwarzchildMetric( mass, radius, theta )
    metric2 = ShwarzchildMetric( mass, radius, theta )
    metric3 = ShwarzchildMetric( mass, radius, theta )
    result = 0.0
    for i in range( 4 ):
        metric1 = DeriveCoordinate( metric1, b, i, symboles[ c ] )
        metric2 = DeriveCoordinate( metric2, c, i, symboles[ b ] )
        metric3 = DeriveCoordinate( metric3, b, c, symboles[ i ] )
        result += 0.5 * metric0[ a ][ i ] * ( metric1[ b ][ i ] + metric2[ c ][ i ] - metric3[ b ][ c ] )
    return result


def DeriveMetricParameter( metric, x, y, withRespectTo ):
    if( type( metric[ x ][ y ] ) != float ):
        return metric[ x ][ y ].diff( withRespectTo )
    return 0.0

def DebugPrint( toPrint, debug ):
    if debug == True:
        if isinstance( toPrint, tuple ):
            print( *toPrint )
        else:
            print( toPrint )

def Connection( mass, radius, theta, a, b, c, symboles, debug = True ):
    global DeriveMetricParameter
    Derive = DeriveMetricParameter
    metric = ShwarzchildMetric( mass, radius, theta )
    result = [ [ 0.0, 0.0, 0.0, 0.0 ], 
                [ 0.0, 0.0, 0.0, 0.0 ], 
                [ 0.0, 0.0, 0.0, 0.0 ], 
                [ 0.0, 0.0, 0.0, 0.0 ] ]
    results = []
    results.append( copy.deepcopy( result ) )
    results.append( copy.deepcopy( result ) )
    results.append( copy.deepcopy( result ) )
    results.append( copy.deepcopy( result ) )
    for l in range( 4 ):
        DebugPrint( '<matrix' + str( l ) + '>', debug )
        for m in range( 4 ):
            DebugPrint( '\t<row' + str( m ) + '>', debug )
            for n in range( 4 ):
                DebugPrint( '\t\t<number' + str( m ) + 'x' + str( n ) + '>', debug )
                for p in range ( 4 ):
                    #print( "Deriving0: ", symboles[ n ], ": ", metric[ m ][ p ], "::", Derive( metric, m, p, symboles[ n ] ) )
                    #print( "Deriving1: ", symboles[ m ], ": ", metric[ n ][ p ], "::", Derive( metric, n, p, symboles[ m ] ) )
                    #print( "Deriving2: ", symboles[ p ], ": ",  metric[ m ][ n ], "::",Derive( metric, m, n, symboles[ p ] ) )
                    DebugPrint( ( '\t\t\t', 0.5 * metric[ l ][ p ], ' * ( ', 
                                               Derive( metric, m, p, symboles[ n ] ), '+', 
                                               Derive( metric, n, p, symboles[ m ] ), '-', 
                                               Derive( metric, m, n, symboles[ p ] ), ' )', 'l: ', l, ' m: ', m, ' n: ', n, ' p: ', p ), debug )
                    results[ l ][ m ][ n ] += ( 0.5 * metric[ l ][ p ] * 
                                               ( Derive( metric, m, p, symboles[ n ] ) + 
                                               Derive( metric, n, p, symboles[ m ] ) - 
                                               Derive( metric, m, n, symboles[ p ] ) ) )
                DebugPrint( ( '\t\t\tresult: ', results[ l ][ m ][ n ], '\n' ), debug )
                DebugPrint( '\t\t\t<ncurrently>', debug )
                for o in results[ l ]:
                    DebugPrint( ( '\t\t\t\t', o ), debug )
                DebugPrint( '\t\t\t</ncurrently>', debug )
                DebugPrint( '\t\t</number' + str( m ) + 'x' + str( n ) + '>', debug )
            DebugPrint( '\t\t<rcurrently>', debug )
            #for i in range( 4 ):
            #    results[ i ] = ( 0.5 * Matrix( metric ) * Matrix( results[ i ] ) ).tolist()
            for o in results[ l ]:
                DebugPrint( ( '\t\t\t', o ), debug )
            DebugPrint( '\t\t</rcurrently>', debug )
            DebugPrint( '\t</row' + str( m ) + '>', debug )
        DebugPrint( '</matrix' + str( l ) + '>\n\n', debug )
    return results

In [64]:
print( CoordinateString( 3, 1, 3 ), IntermediateConnection( 1.0, radius, theta, 3, 1, 3, symboles ) )
print( CoordinateString( 3, 2, 3 ), IntermediateConnection( 1.0, radius, theta, 3, 2, 3, symboles ) )
print( CoordinateString( 2, 2, 2 ), IntermediateConnection( 1.0, radius, theta, 2, 2, 2, symboles ) )
print( CoordinateString( 2, 1, 3 ), IntermediateConnection( 1.0, radius, theta, 2, 1, 3, symboles ) )
#print( CoordinateString( 3, 1, 3 ), IntermediateConnection( 1.0, radius, theta, 3, 1, 3, symboles ) )
#print( CoordinateString( 3, 2, 3 ), IntermediateConnection( 1.0, radius, theta, 3, 2, 3, symboles ) )
print( CoordinateString( 0, 0, 3 ), IntermediateConnection( 1.0, radius, theta, 0, 0, 1, symboles ) )
print( CoordinateString( 1, 0, 0 ), IntermediateConnection( 1.0, radius, theta, 1, 0, 0, symboles ) )
print( CoordinateString( 1, 1, 1 ), IntermediateConnection( 1.0, radius, theta, 1, 1, 1, symboles ) )
print( CoordinateString( 1, 3, 3 ), IntermediateConnection( 1.0, radius, theta, 1, 3, 3, symboles ) )
print( CoordinateString( 1, 2, 2 ), IntermediateConnection( 1.0, radius, theta, 1, 2, 2, symboles ) )
print( '-----Direct Calculation-----\n\n\n\n' )
connection = Connection( 1.0, radius, theta, 3, 1, 3, symboles )
print( "\n\n" )
for i in connection:
    for j in i:
        print( simplify( j ) )
    print( '\n\n' )


phi, r, phi:  0.5*radius**2*(radius**2*sin(theta)**2 + 2*radius*sin(theta)**2)*sin(theta)**2
phi, theta, phi:  0.5*radius**2*(radius**2*sin(theta)**2 + 2*radius**2*sin(theta)*cos(theta))*sin(theta)**2
theta, theta, theta:  0.5*radius**2*(radius**2 - 2*radius)
theta, r, phi:  0
t, t, phi:  -0.5*(radius - 2.0)*(-(radius - 2.0)/radius - 1/radius + (radius - 2.0)/radius**2)/radius
r, t, t:  0.5*radius*((radius - 2.0)/radius + 1/radius - (radius - 2.0)/radius**2)/(radius - 2.0)
r, r, r:  0.5*radius*(radius/(radius - 2.0) - radius/(radius - 2.0)**2 + 1/(radius - 2.0))/(radius - 2.0)
r, phi, phi:  0.5*radius*(-radius**2*sin(theta)**2 - 2*radius*sin(theta)**2)/(radius - 2.0)
r, theta, theta:  0.5*radius*(-radius**2 - 2*radius)/(radius - 2.0)
-----Direct Calculation-----




<matrix0>
	<row0>
		<number0x0>
			 -0.5*(radius - 2.0)/radius  * (  0 + 0 - 0  ) l:  0  m:  0  n:  0  p:  0
			 0.0  * (  0.0 + 0.0 - -1/radius + (radius - 2.0)/radius**2  ) l:  0  m:  0  n:  0  p:  1
			 0.0  * (  0.0 + 0

				 [0.5*radius*(1/radius - (radius - 2.0)/radius**2)/(radius - 2.0), 0, 0, 0]
				 [0, 0.5*radius*(-radius/(radius - 2.0)**2 + 1/(radius - 2.0))/(radius - 2.0), 0, 0.0]
				 [0.0, 0.0, 0.0, 0.0]
				 [0.0, 0.0, 0.0, 0.0]
			</ncurrently>
		</number1x2>
		<number1x3>
			 0.0  * (  0.0 + 0.0 - 0.0  ) l:  1  m:  1  n:  3  p:  0
			 0.5*radius/(radius - 2.0)  * (  0 + 0.0 - 0.0  ) l:  1  m:  1  n:  3  p:  1
			 0.0  * (  0.0 + 0.0 - 0.0  ) l:  1  m:  1  n:  3  p:  2
			 0.0  * (  0.0 + 2*radius*sin(theta)**2 - 0.0  ) l:  1  m:  1  n:  3  p:  3
			result:  0 

			<ncurrently>
				 [0.5*radius*(1/radius - (radius - 2.0)/radius**2)/(radius - 2.0), 0, 0, 0]
				 [0, 0.5*radius*(-radius/(radius - 2.0)**2 + 1/(radius - 2.0))/(radius - 2.0), 0, 0]
				 [0.0, 0.0, 0.0, 0.0]
				 [0.0, 0.0, 0.0, 0.0]
			</ncurrently>
		</number1x3>
		<rcurrently>
			 [0.5*radius*(1/radius - (radius - 2.0)/radius**2)/(radius - 2.0), 0, 0, 0]
			 [0, 0.5*radius*(-radius/(radius - 2.0)**2 + 1/(radius - 2.0))/(radius

			 0.5*radius**2  * (  0.0 + 0.0 - 0.0  ) l:  2  m:  1  n:  3  p:  2
			 0.0  * (  0.0 + 2*radius*sin(theta)**2 - 0.0  ) l:  2  m:  1  n:  3  p:  3
			result:  0 

			<ncurrently>
				 [0, 0, 0, 0]
				 [0, 0, 1.0*radius**3, 0]
				 [0.0, 0.0, 0.0, 0.0]
				 [0.0, 0.0, 0.0, 0.0]
			</ncurrently>
		</number1x3>
		<rcurrently>
			 [0, 0, 0, 0]
			 [0, 0, 1.0*radius**3, 0]
			 [0.0, 0.0, 0.0, 0.0]
			 [0.0, 0.0, 0.0, 0.0]
		</rcurrently>
	</row1>
	<row2>
		<number2x0>
			 0.0  * (  0.0 + 0 - 0.0  ) l:  2  m:  2  n:  0  p:  0
			 0.0  * (  0.0 + 0.0 - 0.0  ) l:  2  m:  2  n:  0  p:  1
			 0.5*radius**2  * (  0 + 0.0 - 0.0  ) l:  2  m:  2  n:  0  p:  2
			 0.0  * (  0.0 + 0.0 - 0.0  ) l:  2  m:  2  n:  0  p:  3
			result:  0 

			<ncurrently>
				 [0, 0, 0, 0]
				 [0, 0, 1.0*radius**3, 0]
				 [0, 0.0, 0.0, 0.0]
				 [0.0, 0.0, 0.0, 0.0]
			</ncurrently>
		</number2x0>
		<number2x1>
			 0.0  * (  0.0 + 0.0 - 0.0  ) l:  2  m:  2  n:  1  p:  0
			 0.0  * (  0.0 + 0 - 0.0  ) l:  2  m:  2  n:

In [43]:
def MakeConnection( mass, radius, theta, symboles ):
    #coords = [ 'time', 'radius', 'theta', 'phi' ]
    result = [ [ 0.0, 0.0, 0.0, 0.0 ], 
    [ 0.0, 0.0, 0.0, 0.0 ], 
    [ 0.0, 0.0, 0.0, 0.0 ], 
    [ 0.0, 0.0, 0.0, 0.0 ] ]
    results = []
    results.append( copy.deepcopy( result ) )
    results.append( copy.deepcopy( result ) )
    results.append( copy.deepcopy( result ) )
    results.append( copy.deepcopy( result ) )
    for i in range( 4 ):
        for j in range( 4 ):
            #print( 'x: ' + coords[ i ] + ' y: ' + coords[ j ] )
            for k in range( 4 ):
                results[ k ][ i ][ j ] += IntermediateConnection( mass, radius, theta, k, i, j, symboles )
    return results

In [ ]:
newLine = 0
for i in MakeConnection( 1.0, radius, theta, symboles ):
    print( i )
    for j in i:
        print( j )
        q = j
        for k in q:
            k = simplify( k )
        print( q )
        if newLine >= 3:
            print( '\n\n\n\n\n\n\n\n' )
            newLine = 0
        else:
            newLine += 1

In [20]:
def GenerateDerivedMetric( metric, symboles ):
    global DeriveMetricParameter
    Derive = DeriveMetricParameter
    for i in range( 4 ):
        for j in range( 4 ):
            metric[ i ][ j ] = Derive( metric, i, j, symboles[ j ] )
    return metric

In [25]:
GenerateDerivedMetric( ShwarzchildMetric( 1.0, symboles[ 1 ], symboles[ 2 ] ), symboles )

[[0, 0.0, 0.0, 0.0],
 [0.0, -radius/(radius - 2.0)**2 + 1/(radius - 2.0), 0.0, 0.0],
 [0.0, 0.0, 0, 0.0],
 [0.0, 0.0, 0.0, 0]]

In [46]:
q = Matrix( [ [ 0, 1 ], [ 1, 0 ] ] )

In [58]:
q.tolist()

[[0, 1], [1, 0]]

In [55]:
q.__array__().tolist()

[[0, 1], [1, 0]]

In [66]:
A = Matrix( [ [ 1, 3 ], [ 6, 8 ] ] )
B = Matrix( [ [ 2, 8 ], [ 4, 6 ] ] )
C = Matrix( [ [ 9, 3 ], [ 7, 5 ] ] )

In [67]:
( A * B ) + ( A * C )

Matrix([
[ 44,  44],
[154, 154]])

In [68]:
A * ( B + C )

Matrix([
[ 44,  44],
[154, 154]])